In [ ]:
from kafka import KafkaConsumer
import psycopg2
import json

consumer = KafkaConsumer(
    'lab7-topic',
    bootstrap_servers='kafka_streaming_lab:9092',
    auto_offset_reset='earliest',
    enable_auto_commit=True,
    group_id='lab7-pg',
    value_deserializer=lambda m: json.loads(m.decode('utf-8'))
)

# Połączenie z PostgreSQL
conn = psycopg2.connect(
    dbname='mydb',
    user='myuser',
    password='myuserpass',
    host='postgres',
    port=5432
)
cursor = conn.cursor()

print("Kafka-to-Postgres processor running...")

for msg in consumer:
    data = msg.value
    cursor.execute(
        "INSERT INTO sensor_data (sensor_id, temperature, timestamp) VALUES (%s, %s, %s)",
        (data['sensor_id'], data['temperature'], data['timestamp'])
    )
    conn.commit()
    print(f"Inserted into DB: {data}")
